In [7]:
import os
import csv
import pandas as pd
import numpy as np

In [8]:
dataset_names=[]
X_trains=[]
y_trains=[]
X_tests=[]

x_train = pd.read_csv(f"./Competition_data/{'Dataset_1'}/X_train.csv",header=0)
y_train = pd.read_csv(f"./Competition_data/{'Dataset_1'}/y_train.csv",header=0)
x_test  = pd.read_csv(f"./Competition_data/{'Dataset_1'}/X_test.csv",header=0) 

# One-Hot Encoding
categorical_columns = x_train.columns[7:20]  # 獲取第八到第二十列名稱
X_train_encoded = pd.get_dummies(x_train, columns=categorical_columns)
X_test_encoded = pd.get_dummies(x_test, columns=categorical_columns)
        
# 確保X_train和X_test具有相同的列
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

X_trains.append(X_train_encoded)
y_trains.append(y_train)
X_tests.append(X_test_encoded)


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score

In [13]:
# 分割資料集為訓練和驗證
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_trains[0], y_trains[0], test_size=0.2, random_state=42)

# 準備基模型
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42))
]

# 準備次級模型
meta_model = LogisticRegression()

# 初始化Stacking Classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# 訓練Stacking Classifier
stacking_model.fit(X_train_part, y_train_part)

# 使用驗證集進行預測
y_pred = stacking_model.predict(X_valid)
y_pred_proba = stacking_model.predict_proba(X_valid)[:, 1] 

# 評估模型
accuracy = accuracy_score(y_valid, y_pred)
f1 = f1_score(y_valid, y_pred, average='weighted')
precision = precision_score(y_valid, y_pred, average='weighted')
auc = roc_auc_score(y_valid, y_pred_proba)

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"AUC: {auc:.2f}")

# 使用訓練好的Stacking模型來預測X_test
y_test_pred = stacking_model.predict_proba(X_trains[0])[:, 1] 

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Accuracy: 0.80
F1 Score: 0.78
Precision: 0.78
AUC: 0.85


In [ ]:
y_test_pred

array([0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,